<h3><b>Importation des librairies</b></h1>

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup

In [179]:
url = "https://books.toscrape.com/catalogue/category/books_1/index.html"

reponse = requests.get(url=url)

reponse.status_code

200

In [180]:
def get_book_themes(url:str="https://books.toscrape.com/catalogue/category/books_1/index.html"):
    reponse = requests.get(url=url)
    if reponse.status_code == 200:
        data_str = reponse.content
        data_html = soup(data_str, "html.parser")
        data = [theme.get_text().strip() for theme in data_html.find_all("div", attrs={"class":"side_categories"})[0].find_all("li")[1:]]
        
        return data
    return None


In [181]:
def get_book_link_themes(url:str="https://books.toscrape.com/catalogue/category/books_1/index.html"):
    reponse = requests.get(url=url)
    if reponse.status_code == 200:
        data_str = reponse.content
        data_html = soup(data_str, "html.parser")
        data = ["https://books.toscrape.com/catalogue/category" + link["href"].split("..")[1] for link in data_html.find_all("div", attrs={"class":"side_categories"})[0].find_all("a")[1:]]
        return data
    return None
    

In [182]:
# table categorie
themes = get_book_themes()
links = get_book_link_themes()

df_categorie = pd.DataFrame({"Nom":themes, "Lien":links})
df_categorie.head()

,Nom,Lien
0,Travel,https://books.toscrape.com/catalogue/category/...
1,Mystery,https://books.toscrape.com/catalogue/category/...
2,Historical Fiction,https://books.toscrape.com/catalogue/category/...
3,Sequential Art,https://books.toscrape.com/catalogue/category/...
4,Classics,https://books.toscrape.com/catalogue/category/...


In [183]:
df_categorie = df_categorie.reset_index()
df_categorie = df_categorie.rename(columns={"index":"Id_categorie"})
df_categorie["Id_categorie"] = df_categorie["Id_categorie"] + 1
df_categorie.head()

,Id_categorie,Nom,Lien
0,1,Travel,https://books.toscrape.com/catalogue/category/...
1,2,Mystery,https://books.toscrape.com/catalogue/category/...
2,3,Historical Fiction,https://books.toscrape.com/catalogue/category/...
3,4,Sequential Art,https://books.toscrape.com/catalogue/category/...
4,5,Classics,https://books.toscrape.com/catalogue/category/...


In [199]:
df_categorie.to_csv("../data/categories.csv", sep=",", index=False)

In [185]:
if __name__ == "__main__":
    url = "https://books.toscrape.com/catalogue/category/books_1/index.html"
    data = get_book_link_themes(url)[:2]
    display(data)

['https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html']

Pour chaque lien, je vais aller recuperer les liens de chaque livre qui me permettront de recupérer la photo de livre, son titre, son titre, la quantité qu'il y a en stock, sa référence (UPC) et sa desciption.

<h3><b>Table livre</b></h3>

In [186]:
links = get_book_link_themes()
links[:2]

['https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html']

In [187]:
url = links[0]

reponse = requests.get(url=url)
reponse.status_code

200

In [188]:
get_book_link_themes()[:5]

['https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html']

In [189]:
# On recupere tous les liens des livres pour un thème données
def get_links_books_for_one_theme(url: str):
    reponse = requests.get(url)
    if reponse.status_code == 200:
        data_str = reponse.content
        data_html = soup(data_str, "html.parser")
        links = data_html.find_all("div", attrs={"class":"image_container"})
        data = [link.find("a")["href"].split("../../")[1] for link in links]
        data_clean = ["https://books.toscrape.com/catalogue/" + x.split("../")[1] for x in data]
        return data_clean
                

# On recupère tous les liens pour tous les thèmes
def get_links_books_for_all_theme():
    links_themes = get_book_link_themes()
    data = []
    for link in links_themes:
        links = get_links_books_for_one_theme(url=link)
        data += links
    return data

links_book = get_links_books_for_all_theme()
links_book[:5]

['https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html',
 'https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html',
 'https://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html',
 'https://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html',
 'https://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html']

Pour chaque lien, nous allons maintenant recupérer les informations pour notre table book

In [190]:
def get_infos_one_book(url:str=links_book[21]):
    reponse = requests.get(url)
    if reponse.status_code == 200:
        data_str = reponse.content
        data_html = soup(data_str, "html.parser")
        # recupération de la catégorie
        categorie = data_html.find("ul", attrs={"class":"breadcrumb"}).find("a").find_next("a").find_next("a").get_text()
        # recupération du titre
        titre = data_html.find("div", attrs={"class":"col-sm-6 product_main"}).find("h1").get_text()
        # recupération du prix
        prix = data_html.find("div", attrs={"class":"col-sm-6 product_main"}).find("p").get_text()[1:]
        # recupération du nombre d'exemplaires disponibles
        disponibilite = data_html.find("div", attrs={"class":"col-sm-6 product_main"}).find("p").find_next("p").get_text().strip().split("(")[1]
        nb_livre_disponible = disponibilite.split(" ")[0]
        # recupération du nombre d'étoiles
        try:
            note = data_html.find("p", attrs={"class":"instock availability"}).find_next("p").attrs["class"][1]
        except:
            note = data_html.find("div", attrs={"class":"alert alert-warning"}).find_previous("p")["class"][1]
        
        STR_TO_NUMBER = {"Zero":0, "One":1, "Two":2, "Three":3, "Four":4, "Five":5}
        # recupération de l'url de l'image
        image_url = "https://books.toscrape.com/" + data_html.find("div", attrs={"class":"item active"}).find("img")["src"].split("../../")[1]
        # recupération de la description
        try:
            description = data_html.find("div", attrs={"id":"product_description"}).find_next("p").get_text()
        except:
            description = None
        # recupération du code UPC
        upc = data_html.find("table", attrs={"class":"table table-striped"}).find("td").get_text()
        
        data = {"Nom": titre, "Categorie":categorie, "Prix":prix, "Disponibilite":nb_livre_disponible, 
                "Note":STR_TO_NUMBER[note], "Code_UPC":upc, "Description":description, "Image_url":image_url, "Lien":url}
        
    return data


def get_infos_all_books():
    datas = []
    links_book = get_links_books_for_all_theme()
    for link in links_book:
        df = get_infos_one_book(link)
        datas.append(df)
        
    return datas
    

In [200]:
# get_infos_one_book(url="https://books.toscrape.com/catalogue/alice-in-wonderland-alices-adventures-in-wonderland-1_5/index.html")

df = get_infos_all_books()
df_book = pd.DataFrame(df)
df_book.head(2)

,Nom,Categorie,Prix,Disponibilite,Note,Code_UPC,Description,Image_url,Lien
0,It's Only the Himalayas,Travel,45.17,19,2,a22124811bfa8350,"“Wherever you go, whatever you do, just . . . ...",https://books.toscrape.com/media/cache/6d/41/6...,https://books.toscrape.com/catalogue/its-only-...
1,Full Moon over Noah’s Ark: An Odyssey to Mount...,Travel,49.43,15,4,ce60436f52c5ee68,Acclaimed travel writer Rick Antonson sets his...,https://books.toscrape.com/media/cache/fe/8a/f...,https://books.toscrape.com/catalogue/full-moon...


In [201]:
df_book = df_book.reset_index()
df_book = df_book.rename(columns={"index":"Id_book"})
df_book["Id_book"] = df_book["Id_book"] + 1

df_book.head()

,Id_book,Nom,Categorie,Prix,Disponibilite,Note,Code_UPC,Description,Image_url,Lien
0,1,It's Only the Himalayas,Travel,45.17,19,2,a22124811bfa8350,"“Wherever you go, whatever you do, just . . . ...",https://books.toscrape.com/media/cache/6d/41/6...,https://books.toscrape.com/catalogue/its-only-...
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,Travel,49.43,15,4,ce60436f52c5ee68,Acclaimed travel writer Rick Antonson sets his...,https://books.toscrape.com/media/cache/fe/8a/f...,https://books.toscrape.com/catalogue/full-moon...
2,3,See America: A Celebration of Our National Par...,Travel,48.87,14,3,f9705c362f070608,To coincide with the 2016 centennial anniversa...,https://books.toscrape.com/media/cache/c7/1a/c...,https://books.toscrape.com/catalogue/see-ameri...
3,4,Vagabonding: An Uncommon Guide to the Art of L...,Travel,36.94,8,2,1809259a5a5f1d8d,With a new foreword by Tim Ferriss •There’s no...,https://books.toscrape.com/media/cache/ca/30/c...,https://books.toscrape.com/catalogue/vagabondi...
4,5,Under the Tuscan Sun,Travel,37.33,7,3,a94350ee74deaa07,A CLASSIC FROM THE BESTSELLING AUTHOR OF UNDER...,https://books.toscrape.com/media/cache/45/21/4...,https://books.toscrape.com/catalogue/under-the...


In [202]:
display(df_book.head(2))
display(df_categorie.head(2))

,Id_book,Nom,Categorie,Prix,Disponibilite,Note,Code_UPC,Description,Image_url,Lien
0,1,It's Only the Himalayas,Travel,45.17,19,2,a22124811bfa8350,"“Wherever you go, whatever you do, just . . . ...",https://books.toscrape.com/media/cache/6d/41/6...,https://books.toscrape.com/catalogue/its-only-...
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,Travel,49.43,15,4,ce60436f52c5ee68,Acclaimed travel writer Rick Antonson sets his...,https://books.toscrape.com/media/cache/fe/8a/f...,https://books.toscrape.com/catalogue/full-moon...


,Id_categorie,Nom,Lien
0,1,Travel,https://books.toscrape.com/catalogue/category/...
1,2,Mystery,https://books.toscrape.com/catalogue/category/...


In [203]:
df_books = pd.merge(df_book, df_categorie[["Nom", "Id_categorie"]], how="left", left_on="Categorie", right_on="Nom")
df_books.head(2)

,Id_book,Nom_x,Categorie,Prix,Disponibilite,Note,Code_UPC,Description,Image_url,Lien,Nom_y,Id_categorie
0,1,It's Only the Himalayas,Travel,45.17,19,2,a22124811bfa8350,"“Wherever you go, whatever you do, just . . . ...",https://books.toscrape.com/media/cache/6d/41/6...,https://books.toscrape.com/catalogue/its-only-...,Travel,1
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,Travel,49.43,15,4,ce60436f52c5ee68,Acclaimed travel writer Rick Antonson sets his...,https://books.toscrape.com/media/cache/fe/8a/f...,https://books.toscrape.com/catalogue/full-moon...,Travel,1


In [204]:
df_books = df_books.drop(columns=["Nom_y"])
df_books = df_books.rename(columns={"Nom_x":"Nom"})
df_books = df_books.drop(columns=["Categorie"])
df_books.head(2)

,Id_book,Nom,Prix,Disponibilite,Note,Code_UPC,Description,Image_url,Lien,Id_categorie
0,1,It's Only the Himalayas,45.17,19,2,a22124811bfa8350,"“Wherever you go, whatever you do, just . . . ...",https://books.toscrape.com/media/cache/6d/41/6...,https://books.toscrape.com/catalogue/its-only-...,1
1,2,Full Moon over Noah’s Ark: An Odyssey to Mount...,49.43,15,4,ce60436f52c5ee68,Acclaimed travel writer Rick Antonson sets his...,https://books.toscrape.com/media/cache/fe/8a/f...,https://books.toscrape.com/catalogue/full-moon...,1


In [205]:
df_books.to_csv("../data/books.csv", sep=",", index=False)